In [1]:
import re
import numpy as np

In [2]:
file = open('input.txt')

circuit = {}

for line in file.readlines():
    info = line.split()
    if len(info) == 3:
        target = info[2]
        if bool(re.fullmatch('[0-9]*',info[0])):
            value = np.uint16(info[0])
        else:
            if info[0] not in circuit:
                circuit[info[0]] = np.uint16(0)
            value = circuit[info[0]]
    if len(info) == 4:
        target = info[3]
        if info[1] not in circuit:
            circuit[info[1]] = np.uint16(0)
        value = ~circuit[info[1]]
    if len(info) == 5:
        target = info[4]
        command = info[1]
        if command in ['AND','OR']:
            if info[0] not in circuit:
                circuit[info[0]] = np.uint16(0)
            if info[2] not in circuit:
                circuit[info[2]] = np.uint16(0)
            val1,val2 = circuit[info[0]], circuit[info[2]]
            match command:
                case 'AND':
                    value = val1 & val2
                case 'OR':
                    value = val1 | val2
                case default:
                    print('oh no')
        elif command in ['LSHIFT','RSHIFT']:
            if info[0] not in circuit:
                circuit[info[0]] = np.uint16(0)
            val1 = circuit[info[0]]
            val2 = np.uint16(info[2])
            match command:
                case 'LSHIFT':
                    value = val1<<val2
                case 'RSHIFT':
                    value = val1>>val2
                case default:
                    print("oh no2")
        else:
            print('oops')
    circuit[target] = value

circuit['a']    

    

0

In [3]:
file = open('input.txt')

circuit = {}
operations = ['ASSIGN','COPY','AND','OR','LSHIFT','RSHIFT','NOT']

for operation in operations:
    circuit[operation] = []

for line in file.readlines():
    info = line.split()
    if len(info) == 3:
        target = info[2]
        value = info[0]
        if bool(re.fullmatch('[0-9]*',info[0])):
            operation = 'ASSIGN'
        else:
            operation = 'COPY'
        circuit[operation].append([value,target])

    if len(info) == 4:
        target = info[3]
        value = info[1]
        operation = 'NOT'
        circuit[operation].append([value,target])

    if len(info) == 5:
        target = info[4]
        operation = info[1]
        val1,val2 = info[0],info[2]
        circuit[operation].append([val1,val2,target])
        
for operation in operations:
    circuit[operation] = np.array(circuit[operation])

values = {}

for line in circuit['ASSIGN']:
    values[line[1]] = np.uint16(line[0])

def evaluate(target):
    if bool(re.fullmatch('[0-9]*',target)):
        return np.uint16(target)
    if target in values:
        return values[target]
    else:
        if target in circuit['COPY'][:,1]:
            i = np.argwhere(target == circuit['COPY'][:,1])[0][0]
            value = evaluate(circuit['COPY'][i,0])
        elif target in circuit['NOT'][:,1]:
            i = np.argwhere(target == circuit['NOT'][:,1])[0][0]
            value = ~evaluate(circuit['NOT'][i,0])
        elif target in circuit['LSHIFT'][:,2]:
            i = np.argwhere(target == circuit['LSHIFT'][:,2])[0][0]
            value = evaluate(circuit['LSHIFT'][i,0])<<np.uint16(circuit['LSHIFT'][i,1])
        elif target in circuit['RSHIFT'][:,2]:
            i = np.argwhere(target == circuit['RSHIFT'][:,2])[0][0]
            value = evaluate(circuit['RSHIFT'][i,0])>>np.uint16(circuit['RSHIFT'][i,1])
        elif target in circuit['AND'][:,2]:
            i = np.argwhere(target == circuit['AND'][:,2])[0][0]
            value = evaluate(circuit['AND'][i,0])&evaluate(circuit['AND'][i,1])
        elif target in circuit['OR'][:,2]:
            i = np.argwhere(target == circuit['OR'][:,2])[0][0]
            value = evaluate(circuit['OR'][i,0])|evaluate(circuit['OR'][i,1])
        else:
            print("end for ", target)
        values[target] = value
        return value

In [4]:
solution1 = evaluate('a')
solution1

3176

In [5]:
values = {}
for line in circuit['ASSIGN']:
    values[line[1]] = np.uint16(line[0])

values['b'] = solution1

def evaluate(target):
    if bool(re.fullmatch('[0-9]*',target)):
        return np.uint16(target)
    if target in values:
        return values[target]
    else:
        if target in circuit['COPY'][:,1]:
            i = np.argwhere(target == circuit['COPY'][:,1])[0][0]
            value = evaluate(circuit['COPY'][i,0])
        elif target in circuit['NOT'][:,1]:
            i = np.argwhere(target == circuit['NOT'][:,1])[0][0]
            value = ~evaluate(circuit['NOT'][i,0])
        elif target in circuit['LSHIFT'][:,2]:
            i = np.argwhere(target == circuit['LSHIFT'][:,2])[0][0]
            value = evaluate(circuit['LSHIFT'][i,0])<<np.uint16(circuit['LSHIFT'][i,1])
        elif target in circuit['RSHIFT'][:,2]:
            i = np.argwhere(target == circuit['RSHIFT'][:,2])[0][0]
            value = evaluate(circuit['RSHIFT'][i,0])>>np.uint16(circuit['RSHIFT'][i,1])
        elif target in circuit['AND'][:,2]:
            i = np.argwhere(target == circuit['AND'][:,2])[0][0]
            value = evaluate(circuit['AND'][i,0])&evaluate(circuit['AND'][i,1])
        elif target in circuit['OR'][:,2]:
            i = np.argwhere(target == circuit['OR'][:,2])[0][0]
            value = evaluate(circuit['OR'][i,0])|evaluate(circuit['OR'][i,1])
        else:
            print("end for ", target)
        values[target] = value
        return value

In [6]:
solution2 = evaluate('a')
solution2

14710

In [9]:
len(values)

336